# Imports

In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from google.colab import files

# Helper functions

## Missing value handling

The following three functions are for use in the 'Missing value handling' stage.

**detect_missing_values(df)**
- Input: raw dataframe df
- Output: (1) summary dataframe that contains the number of missing values per column of df , and (2) the total number of missing values within df

**impute_missing_values(df, method)**
- Input: (1) raw dataframe df, and (2) [optionally] the method of imputation ('simple' / 'KNN')
- Output: dataframe df with all missing values imputed by either ('simple') replacing them with the column average, or ('KNN') inferring them using the k nearest neighbours

**remove_missing_values(df)**
- Input: raw dataframe df
- Output: dataframe df with all rows containing missing values removed

In [2]:
def detect_missing_values(df):
    # Count the number of missing values detected in each column
    missing_df = df.isnull().sum()
    missing_df = pd.DataFrame(missing_df)
    missing_df = missing_df.T
    # Count the total number of missing values
    missing_count = int(missing_df.sum().sum())
    return missing_df, missing_count

def impute_missing_values(df, method='simple'):
    df_copy = df.copy()  # Avoid modifying the original DataFrame
    # Select only numeric columns
    num_cols = df_copy.select_dtypes(include=[np.number]).columns

    if method == 'simple':
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    if method == 'KNN':
        imp = KNNImputer(n_neighbors=2, weights="uniform")
    else:
        # Default to the simple imputer
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')

    # Apply imputation only to numeric columns
    df_copy[num_cols] = imp.fit_transform(df_copy[num_cols])
    return df_copy

def remove_missing_values(df):
    df = df[df.notnull().all(axis=1)]
    return df

## Duplicate removal

The following two functions are for use in the 'Duplicate removal' stage.

**detect_duplicates(df)**
- Input: raw dataframe df
- Output: (1) dataframe df with an additional column that contains 'True' if the row is a duplicate, and 'False' otherwise, and (2) the total number of duplicated rows within df

**remove_duplicates(df)**
- Input: raw dataframe df
- Output: dataframe df with all duplicated rows removed

In [3]:
def detect_duplicates(df):
    # Reset previous detection steps
    if 'duplicate' in df.columns:
        df = df.drop('duplicate', axis=1)
    df_copy = df
    # Remove interfering columns
    if 'id' in df_copy.columns:
        df_copy = df_copy.drop('id', axis=1)
    # Add a new column
    df['duplicate'] = df_copy.duplicated(keep='first')
    # Count the number of duplicates detected
    dups_count = df['duplicate'].value_counts().get(True, 0)
    return df, dups_count

def remove_duplicates(df):
    duplicate_df, dups_count = detect_duplicates(df)
    duplicate_df = duplicate_df[duplicate_df.duplicate != True]
    return duplicate_df

## Outlier handling

The following two functions are for use in the 'Outlier handling' stage.

**detect_outliers(df, contamination)**
- Input: (1) raw dataframe df, and (2) a percentage between 0 and 0.5 that determines how many outliers are to be detected
- Output: (1) dataframe df with an additional column that contains 'True' if the row contains an outlier, and 'False' otherwise, and (2) the total number of outliers within df

**remove_outliers(df, contamination)**
- Input: raw dataframe df, and (2) a percentage between 0 and 0.5 that determines how many outliers are to be detected
- Output: dataframe df with all outliers removed

In [12]:
def detect_outliers(data, contamination):
    # Make a copy of the data to retain original categorical labels
    data_original = data.copy()

    # Convert to Pandas DataFrame if necessary
    if not isinstance(data_original, pd.DataFrame):
        data_original = pd.DataFrame(data_original)

    # Convert object columns to numerical for model training
    data_converted = data_original.copy()
    object_cols = data_converted.select_dtypes(include=['object']).columns

    # Check if column is datetime column or categorical column
    for col in object_cols:
        try:
            parsed_col = pd.to_datetime(col, errors='coerce', infer_datetime_format=True)
            timestamp_ratio = parsed_col.notna().mean()  # Proportion of successfully converted values
            if timestamp_ratio > 0.9:  # If most values convert successfully, treat it as a timestamp
                # Convert timestamp column to integer
                year = [int(year_str[:4]) for year_str in data_converted[col]]
                data_converted[col] = year
        except Exception:
            # Treat it as a categorical column
            le = LabelEncoder()
            data_converted[col] = le.fit_transform(data_converted[col])  # Encode categories numerically

    # Train the detection model
    iso = IsolationForest(contamination=contamination)
    yhat = iso.fit_predict(data_converted)

    # Apply predictions to the original DataFrame
    data_original['outlier'] = yhat == -1

    # Count outliers
    outlier_count = data_original['outlier'].sum()

    return data_original, outlier_count

def remove_outliers(df, contamination):
    outlier_df, outlier_count = detect_outliers(df, contamination)
    outlier_df = outlier_df[outlier_df.outlier != True]
    return outlier_df

## Downloading

**downloadable_data(df)** is a helper function that will be used when downloading the final cleaned data. You will not need to call this function.

In [5]:
def downloadable_data(df):
    if 'duplicate' in df.columns:
        df = df.drop(columns=['duplicate'])
    if 'outlier' in df.columns:
        df = df.drop(columns=['outlier'])
    return df

# Data loading

**Please do not modify the below code**

In [6]:
url_car = 'https://github.com/KlaraKramer/data-engineering-datasets/raw/main/corrupted-datasets/corrupted_car.csv'
url_energy = 'https://github.com/KlaraKramer/data-engineering-datasets/raw/main/corrupted-datasets/corrupted_energy.csv'

car_insurance_df = pd.read_csv(url_car)
energy_df = pd.read_csv(url_energy)

# Convert all column names to lowercase and replace spaces with underscores
car_insurance_df.rename(columns=lambda x: x.lower().replace(' ', '_').replace('-', '_'), inplace=True)
energy_df.rename(columns=lambda x: x.lower().replace(' ', '_').replace('-', '_'), inplace=True)
# Remove all special characters from column names
car_insurance_df.rename(columns=lambda x: x.lower().replace(':', '').replace('$', '').replace('(', '').replace(')', ''), inplace=True)
energy_df.rename(columns=lambda x: x.lower().replace(':', '').replace('$', '').replace('(', '').replace(')', ''), inplace=True)

# This is where your data cleaning work begins

**Please fill out the below three code cells using the functions provided to you above**

## Missing value handling


**-> Modify car_insurance_df and energy_df to not contain missing values.**


Functions you can use are:

**detect_missing_values(df)**

**impute_missing_values(df, method)**

**remove_missing_values(df)**

Feel free to scroll back up to remind yourself of what these functions do.

In [16]:
# Choice of:
# 1: Removing rows with missing values
# or
# 2: Imputing missing values (Replacing them with the average value)
# or
# 3: Imputing missing values (Replacing them with neighbouring values)

## Duplicate removal

**-> Modify car_insurance_df and energy_df to not contain duplicated rows.**

Functions you can use are:

**detect_duplicates(df)**

**remove_duplicates(df)**

Feel free to scroll back up to remind yourself of what these functions do.

## Outlier handling

**-> Modify car_insurance_df and energy_df to not contain outliers.**

Functions you can use are:

**detect_outliers(df, contamination)**

**remove_outliers(df, contamination)**

Feel free to scroll back up to remind yourself of what these functions do.

In [14]:
# Choice of how many outiers to detect (contamination percentage)

# Downloading

**Please do not modify the below code**

In [ ]:
car_insurance_df = downloadable_data(car_insurance_df)
car_insurance_df.to_csv('car_clean.csv', index=False)
files.download('car_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
energy_df = downloadable_data(energy_df)
energy_df.to_csv('energy_clean.csv', index=False)
files.download('energy_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Please now also download the .ipynb file as shown in the video:**


1.   On the top left of your screen, click 'File'
2.   Select 'Download'
3.   Select 'Download .ipynb'



